In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", load_in_4bit=True)

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
def read_context_from_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        context = file.read().replace('\n', ' ')
    return context

#making every line into chunk
def read_context_from_txt_line(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        contexts = [line.strip() for line in file.readlines()]
    return contexts

#divide context into chunks
def divide_context_into_chunks(context, chunk_size):
    chunks = []
    for i in range(0, len(context), chunk_size):
        chunks.append(context[i:i+chunk_size])
    return chunks

# Example .txt file path
txt_file_path = "/content/drive/MyDrive/NLP Data/twentyfive_great_things.txt"

# Read context from .txt file
context = read_context_from_txt(txt_file_path)
chunk_size = 5000  # Adjust the chunk size as per your requirement
context_chunks = divide_context_into_chunks(context, chunk_size)
line_context = read_context_from_txt_line(txt_file_path)

print(line_context)
print(context)
print(context_chunks[0])

In [ ]:
def generate_question(context):

    input_text = "Extract 10 question from this passage, as well as their corresponding answer. The answer must be contained entirely within the above text. Context: " + context
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    output = model.generate(input_ids, num_beams=5, num_beam_groups=5, max_new_tokens=200, diversity_penalty=1.0)
    question = tokenizer.decode(output[0])
    #print(tokenizer.decode(outputs[0]))
    del input_ids

    return question

def generate_answers(question, context):

    input_text = "Question: " + question + "Context: " + context
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    output = model.generate(input_ids, num_beams=7, num_beam_groups=7, max_new_tokens=300, diversity_penalty=1.0)
    answer = tokenizer.decode(output[0])
    del input_ids

    return answer

In [ ]:
#question = generate_question(context)
#answer = generate_answers(question, context)

# 25 great things

output_file_path = "questions_and_answers.txt"
with open(output_file_path, "w", encoding="utf-8") as output_file:
    for context in line_context:
        question = generate_question(context)
        answer = generate_answers(question, context)
        output_file.write("Question: " + question + "\n")
        output_file.write("Answer: " + answer + "\n")
        output_file.write("\n")

In [ ]:
# buggy text
txt_file_path = "/content/drive/MyDrive/NLP Data/buggy.txt"

# Read context from .txt file
context = read_context_from_txt(txt_file_path)
chunk_size = 250  # Adjust the chunk size as per your requirement
context_chunks = divide_context_into_chunks(context, chunk_size)


output_file_path = "buggy.txt"
with open(output_file_path, "w", encoding="utf-8") as output_file:
  for i in range(len(context_chunks)):
    question = generate_question(context_chunks[i])
    answer = generate_answers(question, context_chunks[i])
    output_file.write("Question: " + question + "\n")
    output_file.write("Answer: " + answer + "\n")
    output_file.write("\n")



In [ ]:
#commencement.txt
txt_file_path = "/content/drive/MyDrive/NLP Data/commencement.txt"

# Read context from .txt file
context = read_context_from_txt(txt_file_path)
chunk_size = 260  # Adjust the chunk size as per your requirement
context_chunks = divide_context_into_chunks(context, chunk_size)


output_file_path = "commencement.txt"
with open(output_file_path, "w", encoding="utf-8") as output_file:
  for i in range(len(context_chunks)):
    question = generate_question(context_chunks[i])
    answer = generate_answers(question, context_chunks[i])
    output_file.write("Question: " + question + "\n")
    output_file.write("Answer: " + answer + "\n")
    output_file.write("\n")

In [ ]:


#history.txt
txt_file_path = "/content/drive/MyDrive/NLP Data/history_2.txt"

# Read context from .txt file
context = read_context_from_txt(txt_file_path)
chunk_size = 500  # Adjust the chunk size as per your requirement
context_chunks = divide_context_into_chunks(context, chunk_size)


output_file_path = "history.txt"
with open(output_file_path, "w", encoding="utf-8") as output_file:
  for i in range(len(context_chunks)):
    question = generate_question(context_chunks[i])
    answer = generate_answers(question, context_chunks[i])
    output_file.write("Question: " + question + "\n")
    output_file.write("Answer: " + answer + "\n")
    output_file.write("\n")